Get the movie list

In [11]:
from bs4 import BeautifulSoup
import urllib.request
import re
import nltk
from string import punctuation
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
def UrlToHtml(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    req = urllib.request.Request(url=url, headers=headers)
    html = urllib.request.urlopen(req).read()
    return html
def UrlToWords(url,clean):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
    req = urllib.request.Request(url=url, headers=headers)
    html = urllib.request.urlopen(req).read()
    soup = BeautifulSoup(html, "lxml")
    st = LancasterStemmer()
    [s.extract() for s in soup('script')]
    [s.extract() for s in soup('style')]
    string = soup.get_text()
    tokens = nltk.word_tokenize(string)
    text = [w.lower() for w in tokens]
    
    all_p = punctuation + '--' + '|' +'-' + '~' + '``' + '\""'
    words = []
    for i in text:
        i = re.sub(r'[0-9]|/d+', '', i)
        i = re.sub('\s+', '', i)
        if i != '' and len(i) <= 15 and re.search(r'[A-za-z]', i):
            words.append(i)
            if i in all_p:
                words.remove(i)
    
    if clean == True:
        stop = stopwords.words('english')
        words = [w for w in words if w not in stop]
        words = [w for w in words if len(w)>=2]
        words = [st.stem(word) for word in words]
        
    return words

# get words frequences greater than p 
import collections
def get_top_fre(words, p):
    fre = collections.Counter(words)
    fre = fre.most_common()
    fre = dict(fre)
    t1 = []
    t2 = []
    for k,v in fre.items():
        if v>p:
            t1.append(k)
            t2.append(v)
    #frequences = dict(zip(t1,t2))
    return t1

def get_count(words):
    fre = collections.Counter(words)
    return fre

In [16]:
url = 'https://www.imdb.com/chart/top-english-movies'
html = UrlToHtml(url)

In [32]:
soup = BeautifulSoup(html, "lxml")
movie_list_withtag = soup.find_all('td', attrs={'class':'titleColumn'})

In [43]:
movie_list = []
for x in movie_list_withtag:
    movie_list.append(x.contents[1].get_text())

output_filename = 'Movie_list.txt'
output_file = open(output_filename, 'w', encoding='utf-8')
for y in movie_list:
    output_file.write(y)
    output_file.write('\n')
output_file.close()